In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset
from primal_model import PrimalModel
from primal_train import validate

In [14]:
test_dir_1 = '/home/hank/Datasets/QUASAR/small'
testset_1 = QUASARDataset(test_dir_1,num_graphs=100,remove_self_loops=True)
test_dir_2 = '/home/hank/Datasets/QUASAR/small-test'
testset_2 = QUASARDataset(test_dir_2,num_graphs=100,remove_self_loops=True)
test_dir_3 = '/home/hank/Datasets/QUASAR/N40-100'
testset_3 = QUASARDataset(test_dir_3,num_graphs=100,remove_self_loops=True)
test_dir_4 = '/home/hank/Datasets/QUASAR/N50-100'
testset_4 = QUASARDataset(test_dir_4,num_graphs=100,remove_self_loops=True)
train_dir = '/home/hank/Datasets/QUASAR/N30-1000'
trainset = QUASARDataset(train_dir,num_graphs=1000,remove_self_loops=True)
val_dir = '/home/hank/Datasets/QUASAR/N30-100'
valset = QUASARDataset(val_dir,num_graphs=100,remove_self_loops=True)
DEVICE = torch.device('cpu')

Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.


In [15]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 128
GNN_OUT_DIM = GNN_HIDDEN_DIM
GNN_LAYER = 15
LR = 0.01
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
NUM_EPOCHES = 1000
DROPOUT = 0.5
MLP_LAYER = 2
RESIDUAL = True
BATCHNORM = True
FACTOR = True
model = PrimalModel(
    node_feature_mode=NODE_MODE,
    gnn_type=GNN_TYPE,
    mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
    primal_node_mlp_hidden_dim=GNN_OUT_DIM,primal_node_mlp_output_dim=10,
    node_mlp_num_layers=MLP_LAYER,
    primal_edge_mlp_hidden_dim=GNN_OUT_DIM,primal_edge_mlp_output_dim=10,
    edge_mlp_num_layers=MLP_LAYER,
    dropout_rate=DROPOUT,
    relu_slope=0.1,
    residual=RESIDUAL,
    batchnorm=BATCHNORM,
    factor=FACTOR)
# print(model)
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_True_True_True_801.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_5_True_True_True.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_True_True_True.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_15_True_True_True.pth'))
# model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_9_64_2_True_True_True.pth'))
model.load_state_dict(torch.load('./models/primal_model_N30-1000_SAGE_15_128_2_True_True_True.pth'))
model.double()
BATCH_SIZE = 5
train_acc = validate(model,trainset,DEVICE,batch_size=BATCH_SIZE)
val_acc = validate(model,valset,DEVICE,batch_size=BATCH_SIZE)
test_acc_1 = validate(model,testset_1,DEVICE,batch_size=BATCH_SIZE)
test_acc_2 = validate(model,testset_2,DEVICE,batch_size=BATCH_SIZE)
test_acc_3 = validate(model,testset_3,DEVICE,batch_size=BATCH_SIZE)
test_acc_4 = validate(model,testset_4,DEVICE,batch_size=BATCH_SIZE)
print('Train acc: {:.4f}.'.format(train_acc))
print('Val acc: {:.4f}.'.format(val_acc))
print('Test acc 1: {:.4f}.'.format(test_acc_1))
print('Test acc 2: {:.4f}.'.format(test_acc_2))
print('Test acc 3: {:.4f}.'.format(test_acc_3))
print('Test acc 4: {:.4f}.'.format(test_acc_4))

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE. Residual: True. BatchNorm: True. Factor: True. Resmode: 1.


/home/hank/miniconda3/envs/gnnsdp/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Train acc: 0.6781.
Val acc: 0.7084.
Test acc 1: 0.7644.
Test acc 2: 0.7328.
Test acc 3: 0.7517.
Test acc 4: 0.7315.


In [12]:
from dataset import gen_index_map, flatten_Aty
num_nodes = 1
ud_edges  = torch.triu(
                    torch.ones(num_nodes+1,num_nodes+1) - torch.eye(num_nodes+1)).nonzero().t().contiguous()
ud_edges        = ud_edges.t().tolist()
edge_map        = gen_index_map(ud_edges,num_nodes+1)

b = np.random.rand(8,8)
A = b + b.T
print(A)
v, e = flatten_Aty(A,edge_map)
print(v)
print(e)

a = torch.rand((16,1))
print(a)
print(a.view(4,4))

A = np.random.rand(4,4)
print(A)
print(np.reshape(A,(16,)))

[[0.84126486 1.66289054 0.77924311 0.79788334 1.14701972 1.57561252
  0.23774288 1.0335255 ]
 [1.66289054 0.41654491 1.664809   0.98979742 0.84908494 1.27320907
  0.74318917 0.37843264]
 [0.77924311 1.664809   0.62817578 1.03092902 1.07949187 0.31334236
  1.26428875 1.31925805]
 [0.79788334 0.98979742 1.03092902 0.70343573 1.10954436 0.97311811
  1.22105531 0.62859682]
 [1.14701972 0.84908494 1.07949187 1.10954436 1.40433283 0.81997087
  0.54406743 0.99736633]
 [1.57561252 1.27320907 0.31334236 0.97311811 0.81997087 0.09778169
  0.79441776 1.39913408]
 [0.23774288 0.74318917 1.26428875 1.22105531 0.54406743 0.79441776
  1.43585141 0.88163077]
 [1.0335255  0.37843264 1.31925805 0.62859682 0.99736633 1.39913408
  0.88163077 0.85137892]]
[[0.84126486 1.66289054 0.77924311 0.79788334 0.41654491 1.664809
  0.98979742 0.62817578 1.03092902 0.70343573]
 [1.40433283 0.81997087 0.54406743 0.99736633 0.09778169 0.79441776
  1.39913408 1.43585141 0.88163077 0.85137892]]
[[1.57561252 0.23774288 1.